In [1]:
import numpy as np
import pandas as pd

In [2]:
   
def get_video_labels( path_subset , mode = 'label'):
    if mode == 'input':
        names = ["video_id"]
    elif mode == 'label':
        names = ["video_id","label"]
    df = pd.read_csv(path_subset, sep=";" ,names = names)                                       
    if mode == 'label':
        df = df[df.label.isin(labels)]
    return df
                                            
        

In [3]:
data_root = "/home/arneshbatra2212/Desktop/AI/GestureControl/V2"
csv_labels = "/home/arneshbatra2212/Desktop/AI/GestureControl/V2/labels.csv"
csv_train = "/home/arneshbatra2212/Desktop/AI/GestureControl/V2/train.csv"
csv_test = "/home/arneshbatra2212/Desktop/AI/GestureControl/V2/test.csv"
csv_val = "/home/arneshbatra2212/Desktop/AI/GestureControl/V2/validation.csv"
data_vid = "/home/arneshbatra2212/Desktop/DATA/gesture_data/20bn-jester-v1"


labels_df = pd.read_csv(csv_labels , header = None)
labels = [str(label[0]) for label in labels_df.values]
n_labels = len(labels)
#create dictionary
labels_to_int = dict(zip(labels,range(n_labels)))
int_to_labels = dict(zip(range(n_labels),labels))

In [4]:

train_df = get_video_labels(csv_train)
test_df = get_video_labels(csv_test, mode = 'input')
val_df = get_video_labels(csv_val )

In [5]:

#train_df = train_df.astype('str')
#train_df

In [6]:
model_name = "resnet_3d_model"
target_size = (64,95)
nb_frames = 16
skip = 1
batch_size = 32
input_shape = (nb_frames,) + target_size + (3,)


print(input_shape)



(16, 64, 95, 3)


In [7]:
import tensorflow as tf
from image import ImageDataGenerator
datagen=ImageDataGenerator(rescale = 1./255)


2023-09-09 22:45:08.721375: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-09 22:45:08.762924: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-09 22:45:09.369328: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/arneshbatra2212/miniconda3/envs/tensor/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} 

In [8]:
gen_train = datagen.flow_video_from_dataframe(
    train_df,
    directory=data_vid,
    x_col='video_id',
    y_col='label',
    target_size=target_size,
    path_classes=csv_labels,

    batch_size=batch_size,
    nb_frames=nb_frames,
    skip=skip,
    has_ext = True
    
    
)

gen_val = datagen.flow_video_from_dataframe(
    val_df,
    directory=data_vid,
    x_col='video_id',
    y_col='label',
    target_size=target_size,
    path_classes=csv_labels,

    batch_size=batch_size,
    nb_frames=nb_frames,
    skip=skip,
    has_ext = True
    
    
)

Found 118562 video folders belonging to 27 classes.
Found 14787 video folders belonging to 27 classes.


In [9]:
from resnet_model import Resnet3DBuilder

In [10]:
nb_classes = 27
resnet_model = Resnet3DBuilder.build_resnet_101(input_shape , nb_classes , drop_rate=0.5) 

2023-09-09 22:45:12.623022: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-09 22:45:12.643504: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-09 22:45:12.643692: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [11]:
#summary = resnet_model.summary()

In [12]:
from tensorflow.keras.optimizers import SGD

optimizer = SGD(learning_rate=0.001, momentum=0.9, nesterov=False)
resnet_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model_file = "/home/arneshbatra2212/Desktop/AI/GestureControl/V2/resnet_3d_model/resnetmodel.hdf5"

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint(model_file, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [14]:
#callbacks and graphs
import datetime
log_dir = "graphs" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [15]:


from math import ceil


In [ ]:
data = resnet_model.fit(
    gen_train, steps_per_epoch=ceil(len(train_df.video_id)/batch_size), epochs=100, validation_data=gen_val,
    validation_steps=30 ,shuffle=True,  workers = 8 ,use_multiprocessing= False , max_queue_size=20,
    verbose=1, callbacks=[model_checkpoint, tensorboard_callback]
)

Epoch 1/100
